In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.1 MB 8.3 MB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 


In [2]:
from pyomo.environ import *
import numpy as np

Optimization problem:

$ \text{min}\ x_1-2x_2+3x_3 \\ \text{s.t. }\ x_1 \geq 1, x_2 \geq 2, x_3 \geq 1, \\ 2x_1 + x_2 + 2x_3 \leq 10, \\ -x_1 + x_2 + 2x_3 \leq -3, \\ 2x_1 - 3x_2 + x_3 \leq -6$

In [3]:
model = ConcreteModel()

In [4]:
# number of decision variable
N = 3 

#number of constraint 
M = 3

obj_coeff = [1,-2,3]

constr_coeff = np.array([np.array([2,1,2]),np.array([-1,1,2]),np.array([2,-3,1])])
constr_rhs = np.array([10,-3,-6])

l_bd = [1,2,1]
u_bd = [np.inf,np.inf,np.inf]

row_indices = np.arange(M)
col_indices = np.arange(N)

model.x = Var(col_indices)
model.constraints = ConstraintList()



In [5]:
for j in col_indices:
  model.x[j].setlb(l_bd[j])
  model.x[j].setub(u_bd[j])


In [6]:
for i in row_indices:
  model.constraints.add(summation(constr_coeff[i],model.x) <= constr_rhs[i])


In [7]:
model.objective = Objective(expr = summation(obj_coeff,model.x), sense = minimize)

model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     2 :  None :   inf : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraints : Size=3, Index=constraints_index, Active=True
        Key : Lower : Body                     : Upper : Active
          1 :  -Inf :   2*x[0] + x[1] + 2*x[2] :  1

**1(a) Solving optimization problem using glpk solver**

In [8]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 148486 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [9]:
opt = SolverFactory('glpk', executable='/usr/bin/glpsol')
result = opt.solve(model)
print(result)




Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 10
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: other
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.01425933837890625



In [10]:
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: other


Here solver status is given which means it termainated normally and solver termination condition is other this doesn't specify the termination condition whether it was unbounded or infeasible 


**1(b) Solving optimization problem using cbc solver**

In [11]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 148689 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [12]:
opt_cbc = SolverFactory('cbc')

In [13]:
result2 = opt_cbc.solve(model)
print(result2)


    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>

Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.015167236328125



In [14]:
print('Solver status:', result2.solver.status)
print('Solver termination condition:',result2.solver.termination_condition)
print('Solver time:',result2.solver.time)
print("Termination message:", result2.solver.termination_message)

Solver status: warning
Solver termination condition: infeasible
Solver time: 0.015167236328125


Here solver status is warning because the problem didn't ended with a feasible solution or unbounded solution
Solver termination condition specify the reason for termination it states that the solution to the given is not feasible 

**2.a change the optimization problem to a maximization problem and re-solve**


In [15]:
#declaring new objective 
model.new_objective = Objective(expr = summation(obj_coeff,model.x), sense =maximize)

#deactiving previous objective
model.objective.deactivate()

In [16]:
result_max = opt_cbc.solve(model)
print(result_max)

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>

Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: None
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: maximize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.021076679229736328



In [35]:
print('Solver status:', result_max.solver.status)
print('Solver termination condition:',result_max.solver.termination_condition)
print('Solver time:',result_max.solver.time)
print("Termination message:", result_max.solver.termination_message)


Solver status: warning
Solver termination condition: infeasible
Solver time: 0.021076679229736328
Termination message: Model was proven to be infeasible.


The termination condition for this case infeasible solution

Now deactivating changes made in model

In [18]:
# deactivating maximize objective
model.new_objective.deactivate()

In [19]:
model.objective.activate()

**b. Add an upper bound on $ x_2$ as $ x_2 \leq 8 $ and re-solve**


In [20]:
model.x[1].setub(8)
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     2 :  None :     8 : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

2 Objective Declarations
    new_objective : Size=1, Index=None, Active=False
        Key  : Active : Sense    : Expression
        None :  False : maximize : x[0] - 2.0*x[1] + 3*x[2]
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraints : S

In [21]:
result_b = opt_cbc.solve(model)
print(result_b)

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>

Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.01862049102783203



In [36]:
print('Solver status:', result_b.solver.status)
print('Solver termination condition:',result_b.solver.termination_condition)
print('Solver time:',result_b.solver.time)
print("Termination message:", result_b.solver.termination_message)

Solver status: warning
Solver termination condition: infeasible
Solver time: 0.01862049102783203
Termination message: Model was proven to be infeasible.


The solution was infeasible so it terminated.

**Resetting upper bound of $ x_2 $ to $ \infty $**

In [22]:
model.x[1].setub(np.inf)
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     2 :  None :   inf : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

2 Objective Declarations
    new_objective : Size=1, Index=None, Active=False
        Key  : Active : Sense    : Expression
        None :  False : maximize : x[0] - 2.0*x[1] + 3*x[2]
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraints : S

**c. Add a new constraint $ -x_2 + x_3 \leq 9 $ and re-solve.**

In [23]:
model.constraints.add(expr = -model.x[1] + model.x[2] <= 9 )

In [24]:
result_c = opt_cbc.solve(model)
print(result_c)

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>

Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.023634672164916992



In [25]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     2 :  None :   inf : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

2 Objective Declarations
    new_objective : Size=1, Index=None, Active=False
        Key  : Active : Sense    : Expression
        None :  False : maximize : x[0] - 2.0*x[1] + 3*x[2]
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraints 

In [37]:
print('Solver status:', result_c.solver.status)
print('Solver termination condition:',result_c.solver.termination_condition)
print('Solver time:',result_c.solver.time)
print("Termination message:", result_c.solver.termination_message)

Solver status: warning
Solver termination condition: infeasible
Solver time: 0.023634672164916992
Termination message: Model was proven to be infeasible.


The model was infeasible.

**Deactivating constraint added in c**

In [26]:
model.constraints[4].deactivate()

In [27]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     2 :  None :   inf : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

2 Objective Declarations
    new_objective : Size=1, Index=None, Active=False
        Key  : Active : Sense    : Expression
        None :  False : maximize : x[0] - 2.0*x[1] + 3*x[2]
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - 2.0*x[1] + 3*x[2]

1 Constraint Declarations
    constraints 

**d. Deactivate the objective of the problem and construct a new objective of the form min $x_1 + x_2 + x_3 $ and dactivate the constraint $2x_1+x_2+2x_3 \leq 10$
and add a different constraint $ x_1 + x_2 \geq 45$**

In [28]:
model.objective_d = Objective(expr = model.x[0] + model.x[1] + model.x[2], sense = minimize)

In [29]:
model.objective.deactivate()

In [30]:
model.constraints[1].deactivate()

In [31]:
model.constraints.add(expr = model.x[0] + model.x[1] >= 45)

In [32]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   inf : False :  True :  Reals
          1 :     2 :  None :   inf : False :  True :  Reals
          2 :     1 :  None :   inf : False :  True :  Reals

3 Objective Declarations
    new_objective : Size=1, Index=None, Active=False
        Key  : Active : Sense    : Expression
        None :  False : maximize : x[0] - 2.0*x[1] + 3*x[2]
    objective : Size=1, Index=None, Active=False
        Key  : Active : Sense    : Expression
        None :  False : minimize : x[0] - 2.0*x[1] + 3*x[2]
    objective_d : Size=1, Index=None, A

In [33]:
result_d = opt_cbc.solve(model)
print(result_d)


Problem: 
- Name: unknown
  Lower bound: 46.0
  Upper bound: 46.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 2
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 2
  Error rc: 0
  Time: 0.03764820098876953
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [39]:
print("Objective value:", model.objective_d())
for i in col_indices:
  print(model.x[i]," = ",model.x[i]())

Objective value: 46.0
x[0]  =  25.6
x[1]  =  19.4
x[2]  =  1.0
